1 subject. Feature extraction. Classification for each frequency band.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from sklearn.model_selection import GridSearchCV
import csv

#def select_file(file):
    #Lectura mediante pandas
f = "data_preprocessed_python\\s26.dat"
subject_id = 26

dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)
#df_videos
    # Frecuencia de muestreo
    fs = 128
    # Window
    window = "hann"
    # Length of each segment
    # nperseg = 256 por defecto
    # noverlap
    # Por defecto a None, if None : noverlap = nperseg / 2
    # [...]

    # Definicion de bandas
    eeg_bands = {'Delta': (1, 4),
                'Theta': (4, 8),
                'Alpha': (8, 14),
                'Beta': (14, 30),
                'Gamma': (30, 50)}
# FOURIER
import scipy
from scipy import signal

# SE COGE EL PRIMER VÍDEOS, Df_VIDEOS[0]
def do_stft(video, channel):
    # Array 
    x = df_videos[video][channel]

    # Array of the sample frequency, Array of the segment times, STFT of x
    f, t , Zxx = scipy.signal.stft(x, fs, window)

    #print(np.abs(Zxx))
    '''
    plt.pcolormesh(t, f, np.abs(Zxx), vmin=0, shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    '''
    return Zxx

#SEPARACION EN LAS DISTINTAS BANDAS DE FRECUENCIA

def make_bands(Zxx):
    #Obtener valores reales de STFT, solo positivos.
    values = np.absolute(Zxx)

    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)
    #print(fft_freq)

    # Obtención de cada una de las bandas de frecuencia
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = values[freq_ix]

    #print(eeg_band_fft)
    '''
    eeg_band_fft_means = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft_means[band] = np.mean(values[freq_ix])

    #print(eeg_band_fft_means)

    #Graficación de las medias de los datos
    df = pd.DataFrame(columns=['band', 'val'])
    df['band'] = eeg_bands.keys()
    df['val'] = [eeg_band_fft_means[band] for band in eeg_bands]
    ax = df.plot.bar(x='band', y='val', legend=False)
    ax.set_xlabel("EEG band")
    ax.set_ylabel("Mean band Amplitude")
    '''
    #print(df)

    #print("\n\n")

    '''
    print("Shape de Zxx: ", Zxx.shape)
    print("Shape de Delta: ", eeg_band_fft['Delta'].shape)
    print("Shape de Theta: ", eeg_band_fft['Theta'].shape)
    print("Shape de Alpha: ", eeg_band_fft['Alpha'].shape)
    print("Shape de Beta: ", eeg_band_fft['Beta'].shape)
    print("Shape de Gamma: ", eeg_band_fft['Gamma'].shape)
    '''
    return eeg_band_fft

# Ahora hay que hacer las medias sobre cada banda de frecuencia. La media entre cada una de las posiciones de los bloques. Hacer la media entre todas las posiciones 0, 1, 2... 63 de cada uno de los bloques
def channel_freq(eeg_band_fft):
        
        freq = np.zeros((5, 64))        # 5 bandas, 64 posiciones de los datos

        # Se va a recorrer {Delta, Theta, Alpha, Beta, Gamma}
        for band in eeg_bands:

                # Para cada una de las 64 posiciones del array
                for j in range (0, 64):
                        
                        val = []
                        # Se va a recorrer cada uno de los arrays que hay en cada una de las bandas
                        for i in range (0, len(eeg_band_fft[band])):
                                val.append(eeg_band_fft[band][i][j])
                                # Ejemplo : eeg_band_fft['Delta'][0][0*0 + 0]
                        if (band == "Delta"):
                                freq[0][j] = np.mean(val)
                        elif (band == "Theta"):
                                freq[1][j] = np.mean(val)
                        elif (band == "Alpha"):
                                freq[2][j] = np.mean(val)
                        elif (band == "Beta"):
                                freq[3][j] = np.mean(val)
                        elif (band == "Gamma"):
                                freq[4][j] = np.mean(val)
        return(freq)     # En freq tenemos las frecuencias medias de cada una de las bandas

# Recordamos : Estamos tratando los datos relativos al video 0, un canal concreto de un sujeto concreto
#df_file_videos = select_file('d\\s01.dat')
# Obtener la división por bandas de frecuencia de cada uno de los canales del video 0
for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])
# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

aux_delta
# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

aux_theta
#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

aux_alpha
# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

aux_beta
# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

aux_gamma
df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

aux_labels
data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)
data = data_delta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

    # test_size = 0.2 --> 80% datos para entrenamiento, 20% para test
    # max_depth --> The deeper the tree, the more splits it has and it captures more information about the data. En este modelo, valores altos de max_depths tienden a 'overfit' los datos, lo comprobamos en el siguiente bloque
xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear', 'rbf'),
    'C': [0.1, 1.0, 10.0, 100.0]
}

param_distribs_knn = {
    'n_neighbors': [3,5,6,7,8,9,10],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_delta = accuracy_score(yTest, predict)
svm_acc_delta = accuracy_score(yTest, svm_predict)
knn_acc_delta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_delta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_delta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_delta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_delta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_delta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_delta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_delta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_delta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_delta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_delta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_delta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_delta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_delta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_delta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_delta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_delta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_delta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_delta = f1_score(yTest, knn_predict, pos_label="sad")
data = data_theta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_theta = accuracy_score(yTest, predict)
svm_acc_theta = accuracy_score(yTest, svm_predict)
knn_acc_theta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_theta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_theta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_theta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_theta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_theta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_theta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_theta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_theta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_theta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_theta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_theta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_theta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_theta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_theta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_theta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_theta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_theta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_theta = f1_score(yTest, knn_predict, pos_label="sad")
data = data_alpha.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_alpha = accuracy_score(yTest, predict)
svm_acc_alpha = accuracy_score(yTest, svm_predict)
knn_acc_alpha = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_alpha = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_alpha = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_alpha = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_alpha = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_alpha = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_alpha = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_alpha = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_alpha = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_alpha = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_alpha = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_alpha = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_alpha = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_alpha = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_alpha = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_alpha = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_alpha = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_alpha = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_alpha = f1_score(yTest, knn_predict, pos_label="sad")
data = data_beta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_beta = accuracy_score(yTest, predict)
svm_acc_beta = accuracy_score(yTest, svm_predict)
knn_acc_beta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_beta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_beta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_beta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_beta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_beta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_beta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_beta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_beta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_beta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_beta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_beta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_beta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_beta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_beta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_beta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_beta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_beta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_beta = f1_score(yTest, knn_predict, pos_label="sad")
data = data_gamma.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_gamma = accuracy_score(yTest, predict)
svm_acc_gamma = accuracy_score(yTest, svm_predict)
knn_acc_gamma = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_gamma = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_gamma = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_gamma = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_gamma = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_gamma = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_gamma = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_gamma = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_gamma = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_gamma = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_gamma= recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_gamma = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_gamma = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_gamma = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_gamma = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_gamma = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_gamma = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_gamma = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_gamma = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 2, "STFT", "Delta", "-", "Random Forest", rf_acc_delta, rf_prec_happy_delta, rf_prec_sad_delta, rf_rec_happy_delta, rf_rec_sad_delta, rf_f1_happy_delta, rf_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "SVM", svm_acc_delta, svm_prec_happy_delta, svm_prec_sad_delta, svm_rec_happy_delta, svm_rec_sad_delta, svm_f1_happy_delta, svm_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "kNN", knn_acc_delta, knn_prec_happy_delta, knn_prec_sad_delta, knn_rec_happy_delta, knn_rec_sad_delta, knn_f1_happy_delta, knn_f1_sad_delta], [subject_id, 2, "STFT", "Theta", "-", "Random Forest", rf_acc_theta, rf_prec_happy_theta, rf_prec_sad_theta, rf_rec_happy_theta, rf_rec_sad_theta, rf_f1_happy_theta, rf_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "SVM", svm_acc_theta, svm_prec_happy_theta, svm_prec_sad_theta, svm_rec_happy_theta, svm_rec_sad_theta, svm_f1_happy_theta, svm_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "kNN", knn_acc_theta, knn_prec_happy_theta, knn_prec_sad_theta, knn_rec_happy_theta, knn_rec_sad_theta, knn_f1_happy_theta, knn_f1_sad_theta], [subject_id, 2, "STFT", "Alpha", "-", "Random Forest", rf_acc_alpha, rf_prec_happy_alpha, rf_prec_sad_alpha, rf_rec_happy_alpha, rf_rec_sad_alpha, rf_f1_happy_alpha, rf_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "SVM", svm_acc_alpha, svm_prec_happy_alpha, svm_prec_sad_alpha, svm_rec_happy_alpha, svm_rec_sad_alpha, svm_f1_happy_alpha, svm_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "kNN", knn_acc_alpha, knn_prec_happy_alpha, knn_prec_sad_alpha, knn_rec_happy_alpha, knn_rec_sad_alpha, knn_f1_happy_alpha, knn_f1_sad_alpha], [subject_id, 2, "STFT", "Beta", "-", "Random Forest", rf_acc_beta, rf_prec_happy_beta, rf_prec_sad_beta, rf_rec_happy_beta, rf_rec_sad_beta, rf_f1_happy_beta, rf_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "SVM", svm_acc_beta, svm_prec_happy_beta, svm_prec_sad_beta, svm_rec_happy_beta, svm_rec_sad_beta, svm_f1_happy_beta, svm_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "kNN", knn_acc_beta, knn_prec_happy_beta, knn_prec_sad_beta, knn_rec_happy_beta, knn_rec_sad_beta, knn_f1_happy_beta, knn_f1_sad_beta], [subject_id, 2, "STFT", "Gamma", "-", "Random Forest", rf_acc_gamma, rf_prec_happy_gamma, rf_prec_sad_gamma, rf_rec_happy_gamma, rf_rec_sad_gamma, rf_f1_happy_gamma, rf_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "SVM", svm_acc_gamma, svm_prec_happy_gamma, svm_prec_sad_gamma, svm_rec_happy_gamma, svm_rec_sad_gamma, svm_f1_happy_gamma, svm_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "kNN", knn_acc_gamma, knn_prec_happy_gamma, knn_prec_sad_gamma, knn_rec_happy_gamma, knn_rec_sad_gamma, knn_f1_happy_gamma, knn_f1_sad_gamma]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment2_1.csv", mode="a", header=False, index=False)

from sklearn.metrics import recall_score

rf_rec_happy_gamma = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_gamma = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_gamma = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_gamma= recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_gamma = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_gamma = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_gamma = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_gamma = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_gamma = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_gamma = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_gamma = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_gamma = f1_score(yTest, knn_predict, pos_label="sad")

data_CSV = [subject_id, 2, "STFT", "Delta", "-", "Random Forest", rf_acc_delta, rf_prec_happy_delta, rf_prec_sad_delta, rf_rec_happy_delta, rf_rec_sad_delta, rf_f1_happy_delta, rf_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "SVM", svm_acc_delta, svm_prec_happy_delta, svm_prec_sad_delta, svm_rec_happy_delta, svm_rec_sad_delta, svm_f1_happy_delta, svm_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "kNN", knn_acc_delta, knn_prec_happy_delta, knn_prec_sad_delta, knn_rec_happy_delta, knn_rec_sad_delta, knn_f1_happy_delta, knn_f1_sad_delta], [subject_id, 2, "STFT", "Theta", "-", "Random Forest", rf_acc_theta, rf_prec_happy_theta, rf_prec_sad_theta, rf_rec_happy_theta, rf_rec_sad_theta, rf_f1_happy_theta, rf_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "SVM", svm_acc_theta, svm_prec_happy_theta, svm_prec_sad_theta, svm_rec_happy_theta, svm_rec_sad_theta, svm_f1_happy_theta, svm_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "kNN", knn_acc_theta, knn_prec_happy_theta, knn_prec_sad_theta, knn_rec_happy_theta, knn_rec_sad_theta, knn_f1_happy_theta, knn_f1_sad_theta], [subject_id, 2, "STFT", "Alpha", "-", "Random Forest", rf_acc_alpha, rf_prec_happy_alpha, rf_prec_sad_alpha, rf_rec_happy_alpha, rf_rec_sad_alpha, rf_f1_happy_alpha, rf_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "SVM", svm_acc_alpha, svm_prec_happy_alpha, svm_prec_sad_alpha, svm_rec_happy_alpha, svm_rec_sad_alpha, svm_f1_happy_alpha, svm_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "kNN", knn_acc_alpha, knn_prec_happy_alpha, knn_prec_sad_alpha, knn_rec_happy_alpha, knn_rec_sad_alpha, knn_f1_happy_alpha, knn_f1_sad_alpha], [subject_id, 2, "STFT", "Beta", "-", "Random Forest", rf_acc_beta, rf_prec_happy_beta, rf_prec_sad_beta, rf_rec_happy_beta, rf_rec_sad_beta, rf_f1_happy_beta, rf_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "SVM", svm_acc_beta, svm_prec_happy_beta, svm_prec_sad_beta, svm_rec_happy_beta, svm_rec_sad_beta, svm_f1_happy_beta, svm_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "kNN", knn_acc_beta, knn_prec_happy_beta, knn_prec_sad_beta, knn_rec_happy_beta, knn_rec_sad_beta, knn_f1_happy_beta, knn_f1_sad_beta], [subject_id, 2, "STFT", "Gamma", "-", "Random Forest", rf_acc_gamma, rf_prec_happy_gamma, rf_prec_sad_gamma, rf_rec_happy_gamma, rf_rec_sad_gamma, rf_f1_happy_gamma, rf_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "SVM", svm_acc_gamma, svm_prec_happy_gamma, svm_prec_sad_gamma, svm_rec_happy_gamma, svm_rec_sad_gamma, svm_f1_happy_gamma, svm_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "kNN", knn_acc_gamma, knn_prec_happy_gamma, knn_prec_sad_gamma, knn_rec_happy_gamma, knn_rec_sad_gamma, knn_f1_happy_gamma, knn_f1_sad_gamma]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment2_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s27.dat"
subject_id = 27

dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

data = data_delta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

    # test_size = 0.2 --> 80% datos para entrenamiento, 20% para test
    # max_depth --> The deeper the tree, the more splits it has and it captures more information about the data. En este modelo, valores altos de max_depths tienden a 'overfit' los datos, lo comprobamos en el siguiente bloque
xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_delta = accuracy_score(yTest, predict)
svm_acc_delta = accuracy_score(yTest, svm_predict)
knn_acc_delta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_delta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_delta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_delta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_delta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_delta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_delta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_delta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_delta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_delta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_delta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_delta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_delta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_delta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_delta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_delta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_delta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_delta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_delta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_theta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_theta = accuracy_score(yTest, predict)
svm_acc_theta = accuracy_score(yTest, svm_predict)
knn_acc_theta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_theta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_theta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_theta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_theta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_theta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_theta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_theta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_theta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_theta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_theta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_theta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_theta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_theta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_theta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_theta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_theta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_theta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_theta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_alpha.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_alpha = accuracy_score(yTest, predict)
svm_acc_alpha = accuracy_score(yTest, svm_predict)
knn_acc_alpha = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_alpha = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_alpha = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_alpha = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_alpha = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_alpha = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_alpha = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_alpha = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_alpha = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_alpha = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_alpha = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_alpha = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_alpha = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_alpha = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_alpha = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_alpha = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_alpha = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_alpha = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_alpha = f1_score(yTest, knn_predict, pos_label="sad")

data = data_beta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_beta = accuracy_score(yTest, predict)
svm_acc_beta = accuracy_score(yTest, svm_predict)
knn_acc_beta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_beta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_beta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_beta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_beta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_beta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_beta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_beta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_beta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_beta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_beta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_beta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_beta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_beta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_beta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_beta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_beta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_beta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_beta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_gamma.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_gamma = accuracy_score(yTest, predict)
svm_acc_gamma = accuracy_score(yTest, svm_predict)
knn_acc_gamma = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_gamma = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_gamma = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_gamma = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_gamma = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_gamma = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_gamma = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_gamma = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_gamma = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_gamma = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_gamma= recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_gamma = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_gamma = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_gamma = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_gamma = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_gamma = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_gamma = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_gamma = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_gamma = f1_score(yTest, knn_predict, pos_label="sad")

data_CSV = [subject_id, 2, "STFT", "Delta", "-", "Random Forest", rf_acc_delta, rf_prec_happy_delta, rf_prec_sad_delta, rf_rec_happy_delta, rf_rec_sad_delta, rf_f1_happy_delta, rf_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "SVM", svm_acc_delta, svm_prec_happy_delta, svm_prec_sad_delta, svm_rec_happy_delta, svm_rec_sad_delta, svm_f1_happy_delta, svm_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "kNN", knn_acc_delta, knn_prec_happy_delta, knn_prec_sad_delta, knn_rec_happy_delta, knn_rec_sad_delta, knn_f1_happy_delta, knn_f1_sad_delta], [subject_id, 2, "STFT", "Theta", "-", "Random Forest", rf_acc_theta, rf_prec_happy_theta, rf_prec_sad_theta, rf_rec_happy_theta, rf_rec_sad_theta, rf_f1_happy_theta, rf_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "SVM", svm_acc_theta, svm_prec_happy_theta, svm_prec_sad_theta, svm_rec_happy_theta, svm_rec_sad_theta, svm_f1_happy_theta, svm_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "kNN", knn_acc_theta, knn_prec_happy_theta, knn_prec_sad_theta, knn_rec_happy_theta, knn_rec_sad_theta, knn_f1_happy_theta, knn_f1_sad_theta], [subject_id, 2, "STFT", "Alpha", "-", "Random Forest", rf_acc_alpha, rf_prec_happy_alpha, rf_prec_sad_alpha, rf_rec_happy_alpha, rf_rec_sad_alpha, rf_f1_happy_alpha, rf_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "SVM", svm_acc_alpha, svm_prec_happy_alpha, svm_prec_sad_alpha, svm_rec_happy_alpha, svm_rec_sad_alpha, svm_f1_happy_alpha, svm_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "kNN", knn_acc_alpha, knn_prec_happy_alpha, knn_prec_sad_alpha, knn_rec_happy_alpha, knn_rec_sad_alpha, knn_f1_happy_alpha, knn_f1_sad_alpha], [subject_id, 2, "STFT", "Beta", "-", "Random Forest", rf_acc_beta, rf_prec_happy_beta, rf_prec_sad_beta, rf_rec_happy_beta, rf_rec_sad_beta, rf_f1_happy_beta, rf_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "SVM", svm_acc_beta, svm_prec_happy_beta, svm_prec_sad_beta, svm_rec_happy_beta, svm_rec_sad_beta, svm_f1_happy_beta, svm_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "kNN", knn_acc_beta, knn_prec_happy_beta, knn_prec_sad_beta, knn_rec_happy_beta, knn_rec_sad_beta, knn_f1_happy_beta, knn_f1_sad_beta], [subject_id, 2, "STFT", "Gamma", "-", "Random Forest", rf_acc_gamma, rf_prec_happy_gamma, rf_prec_sad_gamma, rf_rec_happy_gamma, rf_rec_sad_gamma, rf_f1_happy_gamma, rf_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "SVM", svm_acc_gamma, svm_prec_happy_gamma, svm_prec_sad_gamma, svm_rec_happy_gamma, svm_rec_sad_gamma, svm_f1_happy_gamma, svm_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "kNN", knn_acc_gamma, knn_prec_happy_gamma, knn_prec_sad_gamma, knn_rec_happy_gamma, knn_rec_sad_gamma, knn_f1_happy_gamma, knn_f1_sad_gamma]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment2_1.csv", mode="a", header=False, index=False)


f = "data_preprocessed_python\\s28.dat"
subject_id = 28

dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

data = data_delta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

    # test_size = 0.2 --> 80% datos para entrenamiento, 20% para test
    # max_depth --> The deeper the tree, the more splits it has and it captures more information about the data. En este modelo, valores altos de max_depths tienden a 'overfit' los datos, lo comprobamos en el siguiente bloque
xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_delta = accuracy_score(yTest, predict)
svm_acc_delta = accuracy_score(yTest, svm_predict)
knn_acc_delta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_delta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_delta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_delta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_delta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_delta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_delta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_delta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_delta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_delta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_delta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_delta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_delta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_delta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_delta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_delta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_delta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_delta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_delta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_theta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_theta = accuracy_score(yTest, predict)
svm_acc_theta = accuracy_score(yTest, svm_predict)
knn_acc_theta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_theta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_theta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_theta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_theta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_theta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_theta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_theta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_theta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_theta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_theta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_theta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_theta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_theta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_theta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_theta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_theta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_theta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_theta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_alpha.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_alpha = accuracy_score(yTest, predict)
svm_acc_alpha = accuracy_score(yTest, svm_predict)
knn_acc_alpha = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_alpha = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_alpha = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_alpha = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_alpha = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_alpha = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_alpha = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_alpha = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_alpha = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_alpha = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_alpha = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_alpha = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_alpha = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_alpha = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_alpha = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_alpha = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_alpha = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_alpha = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_alpha = f1_score(yTest, knn_predict, pos_label="sad")

data = data_beta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_beta = accuracy_score(yTest, predict)
svm_acc_beta = accuracy_score(yTest, svm_predict)
knn_acc_beta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_beta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_beta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_beta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_beta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_beta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_beta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_beta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_beta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_beta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_beta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_beta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_beta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_beta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_beta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_beta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_beta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_beta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_beta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_gamma.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_gamma = accuracy_score(yTest, predict)
svm_acc_gamma = accuracy_score(yTest, svm_predict)
knn_acc_gamma = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_gamma = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_gamma = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_gamma = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_gamma = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_gamma = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_gamma = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_gamma = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_gamma = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_gamma = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_gamma= recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_gamma = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_gamma = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_gamma = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_gamma = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_gamma = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_gamma = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_gamma = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_gamma = f1_score(yTest, knn_predict, pos_label="sad")

data_CSV = [subject_id, 2, "STFT", "Delta", "-", "Random Forest", rf_acc_delta, rf_prec_happy_delta, rf_prec_sad_delta, rf_rec_happy_delta, rf_rec_sad_delta, rf_f1_happy_delta, rf_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "SVM", svm_acc_delta, svm_prec_happy_delta, svm_prec_sad_delta, svm_rec_happy_delta, svm_rec_sad_delta, svm_f1_happy_delta, svm_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "kNN", knn_acc_delta, knn_prec_happy_delta, knn_prec_sad_delta, knn_rec_happy_delta, knn_rec_sad_delta, knn_f1_happy_delta, knn_f1_sad_delta], [subject_id, 2, "STFT", "Theta", "-", "Random Forest", rf_acc_theta, rf_prec_happy_theta, rf_prec_sad_theta, rf_rec_happy_theta, rf_rec_sad_theta, rf_f1_happy_theta, rf_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "SVM", svm_acc_theta, svm_prec_happy_theta, svm_prec_sad_theta, svm_rec_happy_theta, svm_rec_sad_theta, svm_f1_happy_theta, svm_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "kNN", knn_acc_theta, knn_prec_happy_theta, knn_prec_sad_theta, knn_rec_happy_theta, knn_rec_sad_theta, knn_f1_happy_theta, knn_f1_sad_theta], [subject_id, 2, "STFT", "Alpha", "-", "Random Forest", rf_acc_alpha, rf_prec_happy_alpha, rf_prec_sad_alpha, rf_rec_happy_alpha, rf_rec_sad_alpha, rf_f1_happy_alpha, rf_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "SVM", svm_acc_alpha, svm_prec_happy_alpha, svm_prec_sad_alpha, svm_rec_happy_alpha, svm_rec_sad_alpha, svm_f1_happy_alpha, svm_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "kNN", knn_acc_alpha, knn_prec_happy_alpha, knn_prec_sad_alpha, knn_rec_happy_alpha, knn_rec_sad_alpha, knn_f1_happy_alpha, knn_f1_sad_alpha], [subject_id, 2, "STFT", "Beta", "-", "Random Forest", rf_acc_beta, rf_prec_happy_beta, rf_prec_sad_beta, rf_rec_happy_beta, rf_rec_sad_beta, rf_f1_happy_beta, rf_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "SVM", svm_acc_beta, svm_prec_happy_beta, svm_prec_sad_beta, svm_rec_happy_beta, svm_rec_sad_beta, svm_f1_happy_beta, svm_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "kNN", knn_acc_beta, knn_prec_happy_beta, knn_prec_sad_beta, knn_rec_happy_beta, knn_rec_sad_beta, knn_f1_happy_beta, knn_f1_sad_beta], [subject_id, 2, "STFT", "Gamma", "-", "Random Forest", rf_acc_gamma, rf_prec_happy_gamma, rf_prec_sad_gamma, rf_rec_happy_gamma, rf_rec_sad_gamma, rf_f1_happy_gamma, rf_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "SVM", svm_acc_gamma, svm_prec_happy_gamma, svm_prec_sad_gamma, svm_rec_happy_gamma, svm_rec_sad_gamma, svm_f1_happy_gamma, svm_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "kNN", knn_acc_gamma, knn_prec_happy_gamma, knn_prec_sad_gamma, knn_rec_happy_gamma, knn_rec_sad_gamma, knn_f1_happy_gamma, knn_f1_sad_gamma]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment2_1.csv", mode="a", header=False, index=False)


f = "data_preprocessed_python\\s29.dat"
subject_id = 29

dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

data = data_delta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

    # test_size = 0.2 --> 80% datos para entrenamiento, 20% para test
    # max_depth --> The deeper the tree, the more splits it has and it captures more information about the data. En este modelo, valores altos de max_depths tienden a 'overfit' los datos, lo comprobamos en el siguiente bloque
xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_delta = accuracy_score(yTest, predict)
svm_acc_delta = accuracy_score(yTest, svm_predict)
knn_acc_delta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_delta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_delta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_delta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_delta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_delta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_delta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_delta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_delta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_delta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_delta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_delta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_delta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_delta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_delta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_delta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_delta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_delta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_delta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_theta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_theta = accuracy_score(yTest, predict)
svm_acc_theta = accuracy_score(yTest, svm_predict)
knn_acc_theta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_theta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_theta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_theta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_theta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_theta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_theta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_theta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_theta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_theta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_theta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_theta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_theta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_theta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_theta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_theta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_theta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_theta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_theta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_alpha.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_alpha = accuracy_score(yTest, predict)
svm_acc_alpha = accuracy_score(yTest, svm_predict)
knn_acc_alpha = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_alpha = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_alpha = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_alpha = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_alpha = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_alpha = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_alpha = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_alpha = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_alpha = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_alpha = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_alpha = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_alpha = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_alpha = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_alpha = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_alpha = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_alpha = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_alpha = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_alpha = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_alpha = f1_score(yTest, knn_predict, pos_label="sad")

data = data_beta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_beta = accuracy_score(yTest, predict)
svm_acc_beta = accuracy_score(yTest, svm_predict)
knn_acc_beta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_beta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_beta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_beta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_beta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_beta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_beta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_beta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_beta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_beta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_beta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_beta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_beta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_beta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_beta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_beta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_beta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_beta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_beta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_gamma.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_gamma = accuracy_score(yTest, predict)
svm_acc_gamma = accuracy_score(yTest, svm_predict)
knn_acc_gamma = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_gamma = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_gamma = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_gamma = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_gamma = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_gamma = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_gamma = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_gamma = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_gamma = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_gamma = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_gamma= recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_gamma = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_gamma = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_gamma = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_gamma = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_gamma = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_gamma = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_gamma = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_gamma = f1_score(yTest, knn_predict, pos_label="sad")

data_CSV = [subject_id, 2, "STFT", "Delta", "-", "Random Forest", rf_acc_delta, rf_prec_happy_delta, rf_prec_sad_delta, rf_rec_happy_delta, rf_rec_sad_delta, rf_f1_happy_delta, rf_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "SVM", svm_acc_delta, svm_prec_happy_delta, svm_prec_sad_delta, svm_rec_happy_delta, svm_rec_sad_delta, svm_f1_happy_delta, svm_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "kNN", knn_acc_delta, knn_prec_happy_delta, knn_prec_sad_delta, knn_rec_happy_delta, knn_rec_sad_delta, knn_f1_happy_delta, knn_f1_sad_delta], [subject_id, 2, "STFT", "Theta", "-", "Random Forest", rf_acc_theta, rf_prec_happy_theta, rf_prec_sad_theta, rf_rec_happy_theta, rf_rec_sad_theta, rf_f1_happy_theta, rf_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "SVM", svm_acc_theta, svm_prec_happy_theta, svm_prec_sad_theta, svm_rec_happy_theta, svm_rec_sad_theta, svm_f1_happy_theta, svm_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "kNN", knn_acc_theta, knn_prec_happy_theta, knn_prec_sad_theta, knn_rec_happy_theta, knn_rec_sad_theta, knn_f1_happy_theta, knn_f1_sad_theta], [subject_id, 2, "STFT", "Alpha", "-", "Random Forest", rf_acc_alpha, rf_prec_happy_alpha, rf_prec_sad_alpha, rf_rec_happy_alpha, rf_rec_sad_alpha, rf_f1_happy_alpha, rf_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "SVM", svm_acc_alpha, svm_prec_happy_alpha, svm_prec_sad_alpha, svm_rec_happy_alpha, svm_rec_sad_alpha, svm_f1_happy_alpha, svm_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "kNN", knn_acc_alpha, knn_prec_happy_alpha, knn_prec_sad_alpha, knn_rec_happy_alpha, knn_rec_sad_alpha, knn_f1_happy_alpha, knn_f1_sad_alpha], [subject_id, 2, "STFT", "Beta", "-", "Random Forest", rf_acc_beta, rf_prec_happy_beta, rf_prec_sad_beta, rf_rec_happy_beta, rf_rec_sad_beta, rf_f1_happy_beta, rf_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "SVM", svm_acc_beta, svm_prec_happy_beta, svm_prec_sad_beta, svm_rec_happy_beta, svm_rec_sad_beta, svm_f1_happy_beta, svm_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "kNN", knn_acc_beta, knn_prec_happy_beta, knn_prec_sad_beta, knn_rec_happy_beta, knn_rec_sad_beta, knn_f1_happy_beta, knn_f1_sad_beta], [subject_id, 2, "STFT", "Gamma", "-", "Random Forest", rf_acc_gamma, rf_prec_happy_gamma, rf_prec_sad_gamma, rf_rec_happy_gamma, rf_rec_sad_gamma, rf_f1_happy_gamma, rf_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "SVM", svm_acc_gamma, svm_prec_happy_gamma, svm_prec_sad_gamma, svm_rec_happy_gamma, svm_rec_sad_gamma, svm_f1_happy_gamma, svm_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "kNN", knn_acc_gamma, knn_prec_happy_gamma, knn_prec_sad_gamma, knn_rec_happy_gamma, knn_rec_sad_gamma, knn_f1_happy_gamma, knn_f1_sad_gamma]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment2_1.csv", mode="a", header=False, index=False)


f = "data_preprocessed_python\\s30.dat"
subject_id = 30

dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

data = data_delta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

    # test_size = 0.2 --> 80% datos para entrenamiento, 20% para test
    # max_depth --> The deeper the tree, the more splits it has and it captures more information about the data. En este modelo, valores altos de max_depths tienden a 'overfit' los datos, lo comprobamos en el siguiente bloque
xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_delta = accuracy_score(yTest, predict)
svm_acc_delta = accuracy_score(yTest, svm_predict)
knn_acc_delta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_delta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_delta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_delta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_delta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_delta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_delta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_delta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_delta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_delta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_delta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_delta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_delta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_delta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_delta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_delta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_delta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_delta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_delta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_theta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_theta = accuracy_score(yTest, predict)
svm_acc_theta = accuracy_score(yTest, svm_predict)
knn_acc_theta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_theta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_theta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_theta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_theta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_theta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_theta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_theta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_theta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_theta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_theta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_theta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_theta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_theta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_theta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_theta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_theta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_theta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_theta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_alpha.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_alpha = accuracy_score(yTest, predict)
svm_acc_alpha = accuracy_score(yTest, svm_predict)
knn_acc_alpha = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_alpha = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_alpha = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_alpha = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_alpha = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_alpha = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_alpha = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_alpha = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_alpha = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_alpha = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_alpha = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_alpha = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_alpha = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_alpha = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_alpha = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_alpha = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_alpha = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_alpha = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_alpha = f1_score(yTest, knn_predict, pos_label="sad")

data = data_beta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_beta = accuracy_score(yTest, predict)
svm_acc_beta = accuracy_score(yTest, svm_predict)
knn_acc_beta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_beta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_beta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_beta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_beta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_beta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_beta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_beta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_beta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_beta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_beta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_beta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_beta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_beta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_beta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_beta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_beta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_beta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_beta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_gamma.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_gamma = accuracy_score(yTest, predict)
svm_acc_gamma = accuracy_score(yTest, svm_predict)
knn_acc_gamma = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_gamma = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_gamma = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_gamma = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_gamma = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_gamma = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_gamma = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_gamma = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_gamma = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_gamma = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_gamma= recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_gamma = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_gamma = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_gamma = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_gamma = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_gamma = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_gamma = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_gamma = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_gamma = f1_score(yTest, knn_predict, pos_label="sad")

data_CSV = [subject_id, 2, "STFT", "Delta", "-", "Random Forest", rf_acc_delta, rf_prec_happy_delta, rf_prec_sad_delta, rf_rec_happy_delta, rf_rec_sad_delta, rf_f1_happy_delta, rf_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "SVM", svm_acc_delta, svm_prec_happy_delta, svm_prec_sad_delta, svm_rec_happy_delta, svm_rec_sad_delta, svm_f1_happy_delta, svm_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "kNN", knn_acc_delta, knn_prec_happy_delta, knn_prec_sad_delta, knn_rec_happy_delta, knn_rec_sad_delta, knn_f1_happy_delta, knn_f1_sad_delta], [subject_id, 2, "STFT", "Theta", "-", "Random Forest", rf_acc_theta, rf_prec_happy_theta, rf_prec_sad_theta, rf_rec_happy_theta, rf_rec_sad_theta, rf_f1_happy_theta, rf_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "SVM", svm_acc_theta, svm_prec_happy_theta, svm_prec_sad_theta, svm_rec_happy_theta, svm_rec_sad_theta, svm_f1_happy_theta, svm_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "kNN", knn_acc_theta, knn_prec_happy_theta, knn_prec_sad_theta, knn_rec_happy_theta, knn_rec_sad_theta, knn_f1_happy_theta, knn_f1_sad_theta], [subject_id, 2, "STFT", "Alpha", "-", "Random Forest", rf_acc_alpha, rf_prec_happy_alpha, rf_prec_sad_alpha, rf_rec_happy_alpha, rf_rec_sad_alpha, rf_f1_happy_alpha, rf_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "SVM", svm_acc_alpha, svm_prec_happy_alpha, svm_prec_sad_alpha, svm_rec_happy_alpha, svm_rec_sad_alpha, svm_f1_happy_alpha, svm_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "kNN", knn_acc_alpha, knn_prec_happy_alpha, knn_prec_sad_alpha, knn_rec_happy_alpha, knn_rec_sad_alpha, knn_f1_happy_alpha, knn_f1_sad_alpha], [subject_id, 2, "STFT", "Beta", "-", "Random Forest", rf_acc_beta, rf_prec_happy_beta, rf_prec_sad_beta, rf_rec_happy_beta, rf_rec_sad_beta, rf_f1_happy_beta, rf_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "SVM", svm_acc_beta, svm_prec_happy_beta, svm_prec_sad_beta, svm_rec_happy_beta, svm_rec_sad_beta, svm_f1_happy_beta, svm_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "kNN", knn_acc_beta, knn_prec_happy_beta, knn_prec_sad_beta, knn_rec_happy_beta, knn_rec_sad_beta, knn_f1_happy_beta, knn_f1_sad_beta], [subject_id, 2, "STFT", "Gamma", "-", "Random Forest", rf_acc_gamma, rf_prec_happy_gamma, rf_prec_sad_gamma, rf_rec_happy_gamma, rf_rec_sad_gamma, rf_f1_happy_gamma, rf_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "SVM", svm_acc_gamma, svm_prec_happy_gamma, svm_prec_sad_gamma, svm_rec_happy_gamma, svm_rec_sad_gamma, svm_f1_happy_gamma, svm_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "kNN", knn_acc_gamma, knn_prec_happy_gamma, knn_prec_sad_gamma, knn_rec_happy_gamma, knn_rec_sad_gamma, knn_f1_happy_gamma, knn_f1_sad_gamma]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment2_1.csv", mode="a", header=False, index=False)


f = "data_preprocessed_python\\s31.dat"
subject_id = 31

dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

data = data_delta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

    # test_size = 0.2 --> 80% datos para entrenamiento, 20% para test
    # max_depth --> The deeper the tree, the more splits it has and it captures more information about the data. En este modelo, valores altos de max_depths tienden a 'overfit' los datos, lo comprobamos en el siguiente bloque
xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_delta = accuracy_score(yTest, predict)
svm_acc_delta = accuracy_score(yTest, svm_predict)
knn_acc_delta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_delta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_delta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_delta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_delta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_delta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_delta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_delta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_delta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_delta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_delta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_delta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_delta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_delta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_delta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_delta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_delta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_delta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_delta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_theta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_theta = accuracy_score(yTest, predict)
svm_acc_theta = accuracy_score(yTest, svm_predict)
knn_acc_theta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_theta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_theta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_theta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_theta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_theta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_theta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_theta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_theta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_theta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_theta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_theta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_theta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_theta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_theta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_theta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_theta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_theta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_theta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_alpha.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_alpha = accuracy_score(yTest, predict)
svm_acc_alpha = accuracy_score(yTest, svm_predict)
knn_acc_alpha = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_alpha = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_alpha = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_alpha = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_alpha = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_alpha = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_alpha = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_alpha = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_alpha = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_alpha = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_alpha = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_alpha = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_alpha = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_alpha = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_alpha = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_alpha = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_alpha = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_alpha = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_alpha = f1_score(yTest, knn_predict, pos_label="sad")

data = data_beta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_beta = accuracy_score(yTest, predict)
svm_acc_beta = accuracy_score(yTest, svm_predict)
knn_acc_beta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_beta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_beta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_beta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_beta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_beta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_beta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_beta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_beta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_beta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_beta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_beta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_beta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_beta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_beta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_beta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_beta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_beta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_beta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_gamma.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_gamma = accuracy_score(yTest, predict)
svm_acc_gamma = accuracy_score(yTest, svm_predict)
knn_acc_gamma = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_gamma = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_gamma = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_gamma = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_gamma = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_gamma = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_gamma = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_gamma = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_gamma = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_gamma = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_gamma= recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_gamma = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_gamma = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_gamma = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_gamma = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_gamma = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_gamma = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_gamma = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_gamma = f1_score(yTest, knn_predict, pos_label="sad")

data_CSV = [subject_id, 2, "STFT", "Delta", "-", "Random Forest", rf_acc_delta, rf_prec_happy_delta, rf_prec_sad_delta, rf_rec_happy_delta, rf_rec_sad_delta, rf_f1_happy_delta, rf_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "SVM", svm_acc_delta, svm_prec_happy_delta, svm_prec_sad_delta, svm_rec_happy_delta, svm_rec_sad_delta, svm_f1_happy_delta, svm_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "kNN", knn_acc_delta, knn_prec_happy_delta, knn_prec_sad_delta, knn_rec_happy_delta, knn_rec_sad_delta, knn_f1_happy_delta, knn_f1_sad_delta], [subject_id, 2, "STFT", "Theta", "-", "Random Forest", rf_acc_theta, rf_prec_happy_theta, rf_prec_sad_theta, rf_rec_happy_theta, rf_rec_sad_theta, rf_f1_happy_theta, rf_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "SVM", svm_acc_theta, svm_prec_happy_theta, svm_prec_sad_theta, svm_rec_happy_theta, svm_rec_sad_theta, svm_f1_happy_theta, svm_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "kNN", knn_acc_theta, knn_prec_happy_theta, knn_prec_sad_theta, knn_rec_happy_theta, knn_rec_sad_theta, knn_f1_happy_theta, knn_f1_sad_theta], [subject_id, 2, "STFT", "Alpha", "-", "Random Forest", rf_acc_alpha, rf_prec_happy_alpha, rf_prec_sad_alpha, rf_rec_happy_alpha, rf_rec_sad_alpha, rf_f1_happy_alpha, rf_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "SVM", svm_acc_alpha, svm_prec_happy_alpha, svm_prec_sad_alpha, svm_rec_happy_alpha, svm_rec_sad_alpha, svm_f1_happy_alpha, svm_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "kNN", knn_acc_alpha, knn_prec_happy_alpha, knn_prec_sad_alpha, knn_rec_happy_alpha, knn_rec_sad_alpha, knn_f1_happy_alpha, knn_f1_sad_alpha], [subject_id, 2, "STFT", "Beta", "-", "Random Forest", rf_acc_beta, rf_prec_happy_beta, rf_prec_sad_beta, rf_rec_happy_beta, rf_rec_sad_beta, rf_f1_happy_beta, rf_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "SVM", svm_acc_beta, svm_prec_happy_beta, svm_prec_sad_beta, svm_rec_happy_beta, svm_rec_sad_beta, svm_f1_happy_beta, svm_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "kNN", knn_acc_beta, knn_prec_happy_beta, knn_prec_sad_beta, knn_rec_happy_beta, knn_rec_sad_beta, knn_f1_happy_beta, knn_f1_sad_beta], [subject_id, 2, "STFT", "Gamma", "-", "Random Forest", rf_acc_gamma, rf_prec_happy_gamma, rf_prec_sad_gamma, rf_rec_happy_gamma, rf_rec_sad_gamma, rf_f1_happy_gamma, rf_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "SVM", svm_acc_gamma, svm_prec_happy_gamma, svm_prec_sad_gamma, svm_rec_happy_gamma, svm_rec_sad_gamma, svm_f1_happy_gamma, svm_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "kNN", knn_acc_gamma, knn_prec_happy_gamma, knn_prec_sad_gamma, knn_rec_happy_gamma, knn_rec_sad_gamma, knn_f1_happy_gamma, knn_f1_sad_gamma]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment2_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s32.dat"
subject_id = 32

dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

data = data_delta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

    # test_size = 0.2 --> 80% datos para entrenamiento, 20% para test
    # max_depth --> The deeper the tree, the more splits it has and it captures more information about the data. En este modelo, valores altos de max_depths tienden a 'overfit' los datos, lo comprobamos en el siguiente bloque
xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_delta = accuracy_score(yTest, predict)
svm_acc_delta = accuracy_score(yTest, svm_predict)
knn_acc_delta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_delta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_delta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_delta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_delta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_delta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_delta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_delta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_delta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_delta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_delta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_delta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_delta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_delta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_delta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_delta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_delta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_delta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_delta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_theta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_theta = accuracy_score(yTest, predict)
svm_acc_theta = accuracy_score(yTest, svm_predict)
knn_acc_theta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_theta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_theta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_theta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_theta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_theta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_theta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_theta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_theta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_theta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_theta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_theta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_theta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_theta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_theta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_theta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_theta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_theta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_theta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_alpha.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_alpha = accuracy_score(yTest, predict)
svm_acc_alpha = accuracy_score(yTest, svm_predict)
knn_acc_alpha = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_alpha = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_alpha = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_alpha = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_alpha = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_alpha = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_alpha = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_alpha = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_alpha = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_alpha = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_alpha = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_alpha = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_alpha = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_alpha = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_alpha = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_alpha = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_alpha = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_alpha = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_alpha = f1_score(yTest, knn_predict, pos_label="sad")

data = data_beta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_beta = accuracy_score(yTest, predict)
svm_acc_beta = accuracy_score(yTest, svm_predict)
knn_acc_beta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_beta = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_beta = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_beta = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_beta = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_beta = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_beta = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_beta = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_beta = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_beta = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_beta = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_beta = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_beta = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_beta = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_beta = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_beta = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_beta = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_beta = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_beta = f1_score(yTest, knn_predict, pos_label="sad")

data = data_gamma.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment2_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc_gamma = accuracy_score(yTest, predict)
svm_acc_gamma = accuracy_score(yTest, svm_predict)
knn_acc_gamma = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy_gamma = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad_gamma = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy_gamma = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad_gamma = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy_gamma = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad_gamma = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy_gamma = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad_gamma = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy_gamma = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad_gamma= recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy_gamma = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad_gamma = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy_gamma = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad_gamma = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy_gamma = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad_gamma = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy_gamma = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad_gamma = f1_score(yTest, knn_predict, pos_label="sad")

data_CSV = [subject_id, 2, "STFT", "Delta", "-", "Random Forest", rf_acc_delta, rf_prec_happy_delta, rf_prec_sad_delta, rf_rec_happy_delta, rf_rec_sad_delta, rf_f1_happy_delta, rf_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "SVM", svm_acc_delta, svm_prec_happy_delta, svm_prec_sad_delta, svm_rec_happy_delta, svm_rec_sad_delta, svm_f1_happy_delta, svm_f1_sad_delta], [subject_id, 2, "STFT", "Delta", "-", "kNN", knn_acc_delta, knn_prec_happy_delta, knn_prec_sad_delta, knn_rec_happy_delta, knn_rec_sad_delta, knn_f1_happy_delta, knn_f1_sad_delta], [subject_id, 2, "STFT", "Theta", "-", "Random Forest", rf_acc_theta, rf_prec_happy_theta, rf_prec_sad_theta, rf_rec_happy_theta, rf_rec_sad_theta, rf_f1_happy_theta, rf_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "SVM", svm_acc_theta, svm_prec_happy_theta, svm_prec_sad_theta, svm_rec_happy_theta, svm_rec_sad_theta, svm_f1_happy_theta, svm_f1_sad_theta], [subject_id, 2, "STFT", "Theta", "-", "kNN", knn_acc_theta, knn_prec_happy_theta, knn_prec_sad_theta, knn_rec_happy_theta, knn_rec_sad_theta, knn_f1_happy_theta, knn_f1_sad_theta], [subject_id, 2, "STFT", "Alpha", "-", "Random Forest", rf_acc_alpha, rf_prec_happy_alpha, rf_prec_sad_alpha, rf_rec_happy_alpha, rf_rec_sad_alpha, rf_f1_happy_alpha, rf_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "SVM", svm_acc_alpha, svm_prec_happy_alpha, svm_prec_sad_alpha, svm_rec_happy_alpha, svm_rec_sad_alpha, svm_f1_happy_alpha, svm_f1_sad_alpha], [subject_id, 2, "STFT", "Alpha", "-", "kNN", knn_acc_alpha, knn_prec_happy_alpha, knn_prec_sad_alpha, knn_rec_happy_alpha, knn_rec_sad_alpha, knn_f1_happy_alpha, knn_f1_sad_alpha], [subject_id, 2, "STFT", "Beta", "-", "Random Forest", rf_acc_beta, rf_prec_happy_beta, rf_prec_sad_beta, rf_rec_happy_beta, rf_rec_sad_beta, rf_f1_happy_beta, rf_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "SVM", svm_acc_beta, svm_prec_happy_beta, svm_prec_sad_beta, svm_rec_happy_beta, svm_rec_sad_beta, svm_f1_happy_beta, svm_f1_sad_beta], [subject_id, 2, "STFT", "Beta", "-", "kNN", knn_acc_beta, knn_prec_happy_beta, knn_prec_sad_beta, knn_rec_happy_beta, knn_rec_sad_beta, knn_f1_happy_beta, knn_f1_sad_beta], [subject_id, 2, "STFT", "Gamma", "-", "Random Forest", rf_acc_gamma, rf_prec_happy_gamma, rf_prec_sad_gamma, rf_rec_happy_gamma, rf_rec_sad_gamma, rf_f1_happy_gamma, rf_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "SVM", svm_acc_gamma, svm_prec_happy_gamma, svm_prec_sad_gamma, svm_rec_happy_gamma, svm_rec_sad_gamma, svm_f1_happy_gamma, svm_f1_sad_gamma], [subject_id, 2, "STFT", "Gamma", "-", "kNN", knn_acc_gamma, knn_prec_happy_gamma, knn_prec_sad_gamma, knn_rec_happy_gamma, knn_rec_sad_gamma, knn_f1_happy_gamma, knn_f1_sad_gamma]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment2_1.csv", mode="a", header=False, index=False)


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  53 tasks      | elapsed:    2.2s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100